In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pickle

### Se leen los datos. que están guardados en un pickle

Tener los datos guardados como un objeto de Python mejora bastante el tiempo de leída de los mismos, y permite recuperar el DataFrame tal como se requiere trabajar

In [2]:
df = pd.read_pickle('DataFrame.pkl')

In [6]:
pd.Series(df['delito'].unique()).to_csv('delitos.csv')

### Se hace una copia del original y se pasan a lowercase todas las columnas

In [19]:
df_original = df.copy()
df.columns = [column.lower() for column in df.columns]
df.head()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,edad,genero,estado_civil,pais_interno,departamento,ciudad,reincidente,estado_ingreso,actividades_trabajo,actividades_estudio,actividades_enseñanza,nivel_educativo,hijos_menores,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,12/05/11,30/12/09,CO,1972,48,MASCULINO,NaN,REPUBLICA DE COLOMBIA,ANTIOQUIA,ENVIGADO-ANTIOQUIA,SI,Intramuros,SI,SI,NO,CICLO III,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,01/01/10,05/12/11,30/12/09,CO,1962,58,MASCULINO,NaN,REPUBLICA DE COLOMBIA,ANTIOQUIA,ANORI-ANTIOQUIA,SI,Intramuros,NO,NO,NO,CICLO I,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,N,S,S,01/01/10,06/08/13,30/12/09,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja


In [55]:
# Cargo el DF original en caso de hacer algo mal
df = df_original.copy()

In [71]:
# Información de tipo de variable en cada columna y datos faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142499 entries, 0 to 142498
Data columns (total 30 columns):
internoen                 142499 non-null object
delito                    142499 non-null object
tentativa                 142499 non-null object
agravado                  142499 non-null object
calificado                142499 non-null object
fecha_ingreso             142499 non-null datetime64[ns]
fecha_salida              95815 non-null datetime64[ns]
fecha_captura             142499 non-null datetime64[ns]
situacion_juridica        142499 non-null object
ano_nacimiento            142499 non-null int64
edad                      142499 non-null int64
genero                    142499 non-null object
estado_civil              63007 non-null object
pais_interno              139346 non-null object
departamento              142492 non-null object
ciudad                    142499 non-null object
reincidente               142499 non-null object
estado_ingreso            142469 non

### Pasar las columnas de fecha a objetos datetime

In [70]:
columns_fecha = []
for column in df.columns:
    if 'fecha' in column:
        df[column] = pd.to_datetime(df[column])
        columns_fecha.append(column)
df[columns_fecha].head()

,fecha_ingreso,fecha_salida,fecha_captura
0,2010-01-01,2011-12-05,2009-12-30
1,2010-01-01,2011-05-12,2009-12-30
2,2010-01-01,2013-06-08,2009-12-30
3,2010-01-01,2013-06-08,2009-12-30
4,2010-01-01,2013-06-08,2009-12-30


### Modifico la columna de hijos de menores

La columna de hijos menores se modifica para crear una variable categórica. Esto se hizo debido a que son muy pocas las filas que registran un valor numérico en esta columna

In [59]:
tmp_list = list(df['hijos_menores'].unique())
tmp_list.remove('NO'); tmp_list.remove('0')
df['hijos_menores'] = df['hijos_menores'].apply(
    lambda x: 'NO' if (x in ['NO', '0']) else ('SI' if (x in tmp_list) else np.nan)
)

### Imprimo información de columnas

Imprimo la longitud y los rangos de valores de las columnas, para identificar columnas que puedan requerir ser limpiadas o intevenedidas de alguna manera. 

Luego de esto, se decidió que las columnas departamento y ciudad deben ser modificadas.

In [167]:
for column in df.columns:
    print(column, f'- {len(df[column].unique())} valores')
    print(df[column].unique())
    print('\n')
    print('\n')

internoen - 78683 valores
['5496C737C624136DB406CFC6744C684C070A3CD8'
 '1097561F306939C26DB4AC60377063DA7E2B079F'
 'CB7033A707ADE673139F8A17CAD26AD54696C135' ...,
 '254E3CB8B59DFCA30195C2159875D7D4B0F17C3B'
 '436B48AC56103F004F77882635A9D1DA1C5EDAC5'
 'A112E9DC935B0D64DC75852EB0A30D65291F1B64']




delito - 259 valores
['FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO O MUNICIONES'
 'TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES' 'HURTO'
 'UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS' 'LESIONES PERSONALES'
 'ESTAFA' 'ACTO SEXUAL VIOLENTO' 'ACCESO CARNAL VIOLENTO'
 'INASISTENCIA ALIMENTARIA' 'VIOLENCIA INTRAFAMILIAR' 'HOMICIDIO'
 'EXTORSION' 'CONCIERTO PARA DELINQUIR' 'CONCUSION'
 'FABRICACION  TRAFICO Y PORTE DE ARMAS Y MUNICIONES DE USO PRIVATIVO DE LAS FUERZAS ARMADAS'
 'CONSTREÑIMIENTO ILEGAL' 'SUSTRACCION DE BIEN PROPIO'
 'CELEBRACION INDEBIDA DE CONTRATOS DE SEGUROS' 'ABUSO DE CONFIANZA'
 'FALSEDAD EN DOCUMENTO PRIVADO' 'VIOLENCIA CONTRA SERVIDOR PUBLICO'
 'ACTOS SEXUALES CON MENOR 

['ENVIGADO-ANTIOQUIA' 'ANORI-ANTIOQUIA' 'CAJIBIO-CAUCA' ...,
 'MEDIA LUNA-PIVIJAY-MAGDALENA' 'SAN JUAN DE LA CHINA-IBAGUE-TOLIMA'
 'CONDINA GUACARY-PEREIRA']




reincidente - 1 valores
['SI']




estado_ingreso - 7 valores
['Intramuros' 'Prision Domiciliaria' 'Vigilancia Electronica'
 'Detencion Domiciliaria' nan 'Control Electronico' 'Espera Traslado']




actividades_trabajo - 2 valores
['SI' 'NO']




actividades_estudio - 2 valores
['SI' 'NO']




actividades_enseñanza - 2 valores
['NO' 'SI']




nivel_educativo - 12 valores
['CICLO III' 'CICLO I' 'PROFESIONAL' 'CICLO II' 'CICLO IV'
 'TECNICO PROFESIONAL' 'ANALFABETA' 'TECNICO' 'TECNOLOGICO'
 'ESPECIALIZACION' 'POST GRADO' 'MAGISTER']




hijos_menores - 24 valores
['SI' 'NO' '1' '3' '2' '0' '5' '4' '6' '8' '7' '10' '9' '17' '12' '11' '14'
 '16' '13' '21' '19' '24' '23' '20']




condic_expecional - 54 valores
[nan 'AFRO COLOMBIANO' 'MADRE GESTANTE' 'CON DISCAPACIDAD' 'ADULTO MAYOR'
 'INDIGENA' 'TRANSEXUAL' 'BISEXUAL' 'EXTRANJEROS

### Modificación columnas departamento y ciudad

Se crean nuevas columnas con los valores modificados.

In [168]:
df['ciudad_departamento'] = df['ciudad'] + '-' + df['departamento']
df['ciudad_departamento'].replace('--', '-', inplace=True)
df['ciudad_departamento'].replace('–', '-', inplace=True)
df['ciudad_departamento'].replace(',', '-', inplace=True)
df['ciudad_departamento'].replace('.', '-', inplace=True)
df['ciudad_departamento'] = df['ciudad_departamento'].apply(lambda x: str(x).split('-'))
df['ciudad_departamento'] = df['ciudad_departamento'].apply(set)

In [169]:
dpto_list = [
 'RISARALDA',
 'CESAR',
 'SAN ANDRES Y PROVIDENCIA',
 'NORTE DE SANTANDER',
 'CHOCO',
 'SUCRE',
 'CASANARE',
 'QUINDIO',
 'GUAVIARE',
 'ANTIOQUIA',
 'CUNDINAMARCA',
 'PUTUMAYO',
 'NARIÑO',
 'META',
 'HUILA',
 'BOLIVAR',
 'CAUCA',
 'CAQUETA',
 'BOYACA',
 'SANTANDER',
 'CORDOBA',
 'ARAUCA',
 'CALDAS',
 'ATLANTICO',
 'LA GUAJIRA',
 'TOLIMA',
 'VALLE DEL CAUCA',
 'AMAZONAS',
 'MAGDALENA',
 'VAUPES',
 'GUAINIA',
 'VICHADA',   
 'BOGOTA DISTRITO CAPITAL']
len(set(dpto_list))

33

In [158]:
df['ciudad_departamento'] = df['ciudad_departamento'].apply(list)
def f_departamento(lista):
    try:
        for i in lista:
            if i in dpto_list:
                return i
    except (TypeError):
        return np.nan
def f_ciudad(lista):
    try:
        for i in lista:
            if i not in dpto_list:
                return i
        else:
            return np.nan
    except (TypeError):
        return np.nan
df['ciudad_corregido'] = df['ciudad_departamento'].apply(f_ciudad)
df['departamento_corregido'] = df['ciudad_departamento'].apply(f_departamento)
df['departamento_corregido'].value_counts()

ANTIOQUIA                   21561
BOGOTA DISTRITO CAPITAL     19261
VALLE DEL CAUCA             17992
SANTANDER                   10043
TOLIMA                       6626
CALDAS                       6130
CUNDINAMARCA                 5112
HUILA                        4951
CAUCA                        4697
RISARALDA                    4222
NARIÑO                       4171
NORTE DE SANTANDER           3983
BOYACA                       3829
ATLANTICO                    3772
QUINDIO                      3306
BOLIVAR                      3305
META                         2810
CESAR                        2604
CORDOBA                      2268
MAGDALENA                    2203
CAQUETA                      2027
SUCRE                        1704
CHOCO                        1526
LA GUAJIRA                    967
PUTUMAYO                      818
ARAUCA                        659
CASANARE                      650
AMAZONAS                      351
SAN ANDRES Y PROVIDENCIA      351
GUAVIARE      

In [160]:
df.to_pickle('DataFrame_QC1.pkl')

In [2]:
df = pd.read_pickle('DataFrame_QC1.pkl')
pd.options.display.max_columns = 100
df.head()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,edad,genero,estado_civil,pais_interno,departamento,ciudad,reincidente,estado_ingreso,actividades_trabajo,actividades_estudio,actividades_enseñanza,nivel_educativo,hijos_menores,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado,ciudad_departamento,ciudad_corregido,departamento_corregido
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,2010-01-01,2011-12-05,2009-12-30,CO,1972,48,MASCULINO,NaN,REPUBLICA DE COLOMBIA,ANTIOQUIA,ENVIGADO-ANTIOQUIA,SI,Intramuros,SI,SI,NO,CICLO III,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,"[ANTIOQUIA, ENVIGADO]",ENVIGADO,ANTIOQUIA
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,2010-01-01,2011-05-12,2009-12-30,CO,1962,58,MASCULINO,NaN,REPUBLICA DE COLOMBIA,ANTIOQUIA,ANORI-ANTIOQUIA,SI,Intramuros,NO,NO,NO,CICLO I,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,"[ANTIOQUIA, ANORI]",ANORI,ANTIOQUIA
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,2010-01-01,2013-06-08,2009-12-30,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,"[CAUCA, CAJIBIO]",CAJIBIO,CAUCA
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,N,S,S,2010-01-01,2013-06-08,2009-12-30,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,"[CAUCA, CAJIBIO]",CAJIBIO,CAUCA
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,N,N,N,2010-01-01,2013-06-08,2009-12-30,CO,1982,38,MASCULINO,NaN,REPUBLICA DE COLOMBIA,CAUCA,CAJIBIO-CAUCA,SI,Intramuros,SI,SI,NO,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,"[CAUCA, CAJIBIO]",CAJIBIO,CAUCA


In [3]:
df['duracion_encierro'] = (df['fecha_salida'] - df['fecha_ingreso']).apply(lambda x: x.days)
df['duracion_encierro'].head()

0     703.0
1     496.0
2    1254.0
3    1254.0
4    1254.0
Name: duracion_encierro, dtype: float64

In [16]:
df.tail()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,edad,genero,estado_civil,pais_interno,departamento,ciudad,reincidente,estado_ingreso,actividades_trabajo,actividades_estudio,actividades_enseñanza,nivel_educativo,hijos_menores,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado,ciudad_departamento,ciudad_corregido,departamento_corregido,duracion_encierro
142494,A112E9DC935B0D64DC75852EB0A30D65291F1B64,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,2020-05-30,NaT,2003-03-23,CO,1978,41,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,VALLE DEL CAUCA,BUGA-VALLE DEL CAUCA,SI,Intramuros,NO,NO,NO,CICLO II,SI,NaN,227,EPMSC BUGA,VALLE DEL CAUCA,BUGA,OCCIDENTE,Activo,"[BUGA, VALLE DEL CAUCA]",BUGA,VALLE DEL CAUCA,NaN
142495,A112E9DC935B0D64DC75852EB0A30D65291F1B64,HOMICIDIO,N,S,N,2020-05-30,NaT,2003-03-23,CO,1978,41,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,VALLE DEL CAUCA,BUGA-VALLE DEL CAUCA,SI,Intramuros,NO,NO,NO,CICLO II,SI,NaN,227,EPMSC BUGA,VALLE DEL CAUCA,BUGA,OCCIDENTE,Activo,"[BUGA, VALLE DEL CAUCA]",BUGA,VALLE DEL CAUCA,NaN
142496,A112E9DC935B0D64DC75852EB0A30D65291F1B64,HOMICIDIO,S,S,N,2020-05-30,NaT,2003-03-23,CO,1978,41,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,VALLE DEL CAUCA,BUGA-VALLE DEL CAUCA,SI,Intramuros,NO,NO,NO,CICLO II,SI,NaN,227,EPMSC BUGA,VALLE DEL CAUCA,BUGA,OCCIDENTE,Activo,"[BUGA, VALLE DEL CAUCA]",BUGA,VALLE DEL CAUCA,NaN
142497,A112E9DC935B0D64DC75852EB0A30D65291F1B64,HURTO,N,S,N,2020-05-30,NaT,2003-03-23,CO,1978,41,MASCULINO,UNION LIBRE,REPUBLICA DE COLOMBIA,VALLE DEL CAUCA,BUGA-VALLE DEL CAUCA,SI,Intramuros,NO,NO,NO,CICLO II,SI,NaN,227,EPMSC BUGA,VALLE DEL CAUCA,BUGA,OCCIDENTE,Activo,"[BUGA, VALLE DEL CAUCA]",BUGA,VALLE DEL CAUCA,NaN
142498,73CC98628509EC80173897BB76362B1E42658BB5,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,2020-05-30,NaT,2019-06-19,CO,1989,31,FEMENINO,UNION LIBRE,REPUBLICA DE COLOMBIA,CAUCA,POPAYAN-CAUCA,SI,Intramuros,NO,NO,NO,CICLO III,NO,NaN,209,RM POPAYAN,CAUCA,POPAYAN,OCCIDENTE,Activo,"[CAUCA, POPAYAN]",POPAYAN,CAUCA,NaN


In [11]:
print(df['duracion_encierro'] <= 0))
print(sum(df['duracion_encierro'].isna()))

3702
46684


ValueError: Cannot add integral value to Timestamp without freq.

In [103]:
cols_to_drop = ['situacion_juridica', 'reincidente', 'condic_expecional']

189

____________

In [170]:
conteo_reincidencia = df.groupby(['internoen', 'fecha_captura']).count()
conteo_reincidencia = conteo_reincidencia.reset_index()
conteo_reincidencia = conteo_reincidencia[['internoen', 'fecha_captura', 'delito']]
conteo_reincidencia['evento_delictivo'] = conteo_reincidencia['internoen'] + str(conteo_reincidencia['fecha_captura'])
conteo_reincidencia['numero_reincidencia'] = conteo_reincidencia.groupby('internoen').cumcount() + 1
conteo_reincidencia['totdash_appal_reincidencias'] = \
    conteo_reincidencia['internoen'].map(conteo_reincidencia.groupby('internoen')['fecha_captura'].count())
conteo_reincidencia = conteo_reincidencia.rename(columns={'delito': 'conteo_delitos'})
conteo_reincidencia.head(10)

,internoen,fecha_captura,conteo_delitos,evento_delictivo,numero_reincidencia,total_reincidencias
0,000334541B6B3305CE91761942D7B381248777EA,02/08/18,1,000334541B6B3305CE91761942D7B381248777EA0 ...,1,1
1,000357E4F19E3722BA43488F69A7D7DECFC620C4,14/12/17,2,000357E4F19E3722BA43488F69A7D7DECFC620C40 ...,1,1
2,000393AD7FE65FBBE53A61B0EAA1FB9A62F71450,22/04/10,2,000393AD7FE65FBBE53A61B0EAA1FB9A62F714500 ...,1,1
3,0003BE212427DF2E9CCB2449B6012EF915B3FAE3,26/09/14,1,0003BE212427DF2E9CCB2449B6012EF915B3FAE30 ...,1,1
4,00043EB254F6E5EFCE77A8C20BC709B63A4BE443,03/05/19,1,00043EB254F6E5EFCE77A8C20BC709B63A4BE4430 ...,1,1
5,0006F969D7EB7E2DD76A3F7F07D7116798FD51BF,27/06/17,1,0006F969D7EB7E2DD76A3F7F07D7116798FD51BF0 ...,1,1
6,0007F727CB3E6D30F1BFAC3834CDAFAC6FF40025,02/09/16,2,0007F727CB3E6D30F1BFAC3834CDAFAC6FF400250 ...,1,1
7,0008649DEF36F510807D0AB58A84CF74E6083FBF,04/02/20,2,0008649DEF36F510807D0AB58A84CF74E6083FBF0 ...,1,2
8,0008649DEF36F510807D0AB58A84CF74E6083FBF,22/01/16,2,0008649DEF36F510807D0AB58A84CF74E6083FBF0 ...,2,2
9,00086E29AF6E4AD70C7BF9D67FB3A5E2363AC569,21/02/17,1,00086E29AF6E4AD70C7BF9D67FB3A5E2363AC5690 ...,1,1


In [7]:
conteo_reincidencia.describe()

,conteo_delitos,numero_reincidencia,total_reincidencias
count,109323.000000,109323.000000,109323.000000
mean,1.303468,1.369721,1.739442
std,0.671524,0.688707,0.957356
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,2.000000,2.000000
max,13.000000,13.000000,13.000000


In [8]:
serie_tmp = conteo_reincidencia[['evento_delictivo', 'numero_reincidencia']]
serie_tmp.set_index('evento_delictivo', inplace=True)
serie_tmp = serie_tmp['numero_reincidencia']
df['numero_reincidencia'] = df['internoen'] + df['fecha_captura']
df['numero_reincidencia'] = df['numero_reincidencia'].map(serie_tmp)
df.head()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,...,nivel_educativo,hijos_menores,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado,numero_reincidencia
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,12/05/11,30/12/09,CO,1972,...,CICLO III,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,01/01/10,05/12/11,30/12/09,CO,1962,...,CICLO I,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,N,S,S,01/01/10,06/08/13,30/12/09,CO,1982,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,CICLO I,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1


In [9]:
serie_tmp = conteo_reincidencia[['evento_delictivo', 'total_reincidencias']]
serie_tmp.set_index('evento_delictivo', inplace=True)
serie_tmp = serie_tmp['total_reincidencias']
df['total_reincidencias'] = df['internoen'] + df['fecha_captura']
df['total_reincidencias'] = df['total_reincidencias'].map(serie_tmp)
df.head()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,...,hijos_menores,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado,numero_reincidencia,total_reincidencias
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,12/05/11,30/12/09,CO,1972,...,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1,1
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,01/01/10,05/12/11,30/12/09,CO,1962,...,SI,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1,1
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,N,S,S,01/01/10,06/08/13,30/12/09,CO,1982,...,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,SI,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1


In [10]:
serie_tmp = conteo_reincidencia[['evento_delictivo', 'conteo_delitos']]
serie_tmp.set_index('evento_delictivo', inplace=True)
serie_tmp = serie_tmp['conteo_delitos']
df['conteo_delitos'] = df['internoen'] + df['fecha_captura']
df['conteo_delitos'] = df['conteo_delitos'].map(serie_tmp)
df.head()

,internoen,delito,tentativa,agravado,calificado,fecha_ingreso,fecha_salida,fecha_captura,situacion_juridica,ano_nacimiento,...,condic_expecional,codigo_establecimiento,establecimiento,depto_establecimiento,mpio_establecimiento,regional,estado,numero_reincidencia,total_reincidencias,conteo_delitos
0,5496C737C624136DB406CFC6744C684C070A3CD8,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,12/05/11,30/12/09,CO,1972,...,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1,1,1
1,1097561F306939C26DB4AC60377063DA7E2B079F,TRAFICO FABRICACION O PORTE DE ESTUPEFACIENTES,N,N,N,01/01/10,05/12/11,30/12/09,CO,1962,...,NaN,502,EPMSC MEDELLIN,ANTIOQUIA,MEDELLIN,NOROESTE,Baja,1,1,1
2,CB7033A707ADE673139F8A17CAD26AD54696C135,FABRICACION TRAFICO Y PORTE DE ARMAS DE FUEGO ...,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1,3
3,CB7033A707ADE673139F8A17CAD26AD54696C135,HURTO,N,S,S,01/01/10,06/08/13,30/12/09,CO,1982,...,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1,3
4,CB7033A707ADE673139F8A17CAD26AD54696C135,UTILIZACION ILEGAL DE UNIFORMES E INSIGNIAS,N,N,N,01/01/10,06/08/13,30/12/09,CO,1982,...,NaN,235,EPAMSCAS POPAYAN (ERE),CAUCA,POPAYAN,OCCIDENTE,Baja,1,1,3


eventos_delictivos = df.groupby(['internoen', 'fecha_captura']).agg({
    'conteo_delitos': 'mean', 'numero_reincidencia': 'mean', 'genero': 'first'
})
eventos_delictivos.head()